In [21]:
# Импортируем библиотеки
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# Считываем данные из файла
df = pd.read_excel('data.xlsx')

In [23]:
# Удаляем месяц, так как он не несет никакой информации
df = df.drop('month', axis=1)

# Удалим строки с merchant_name, которое встречается меньше 5 раз в df
df = df.groupby('merchant_name').filter(lambda x : len(x) > 5)
df.head(200)

,merchant_name,day,cashback
0,1С Интерес,2022-01-21 00:00:00,240.53
1,1С Интерес,2022-01-22 00:00:00,399.21
2,1С Интерес,2022-01-23 00:00:00,385.44
3,1С Интерес,2022-01-24 00:00:00,209.48
4,1С Интерес,2022-01-25 00:00:00,361.80
...,...,...,...
198,Delta Sirius,2022-01-08 00:00:00,842.06
199,Delta Sirius,2022-01-09 00:00:00,1924.45
200,Delta Sirius,2022-01-10 00:00:00,1897.88
201,Delta Sirius,2022-01-11 00:00:00,3418.17


In [24]:
# Получает на вход год, месяц, день и возвращает день недели в виде его номера
def get_weekday(year, month, day):
    return datetime.date(year, month, day).weekday()

# Получает на вход строку вида "YYYY-MM-DD hh:mm:ss" и возвращает year, month, day
def get_date(date_string):
    date = str(date_string).split(' ')[0]
    year, month, day = map(int, date.split('-'))
    return month, day

# Переведем столбец day формата "YYYY-MM-DD hh:mm:ss" в столбец с результатом get_date
df['day'] = df['day'].apply(lambda x : '.'.join(list(map(str, get_date(x)))))
df['weekday'] = df['day'].apply(lambda x : get_weekday(2022, *map(int, x.split('.'))))
df['is_weekend'] = df['weekday'].apply(lambda x : 1 if x in [5, 6] else 0)

In [25]:
# Для каждого значения merchant_name построим график cashback
for merchant_name in df['merchant_name'].unique():
    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')
    plt.title(merchant_name)
    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')
    plt.close()

In [ ]:
# Сохраним df в xlsx файл для дальнейшего использования
df.to_excel('data.xlsx', index=False)